In [1]:
!pip install peft transformers datasets accelerate bitsandbytes loralib -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
!pip install py7zr -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.8/413.8 kB 26.7 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
dataset = load_dataset("samsum")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/samsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [5]:
from datasets import concatenate_datasets

In [6]:
print(f"Train Dataset Size :{len(dataset['train'])}")
print(f"Validation Dataset Size :{len(dataset['validation'])}")
print(f"Test Dataset Size :{len(dataset['test'])}")

Train Dataset Size :14732
Validation Dataset Size :818
Test Dataset Size :819


In [7]:
model_id = "google/flan-t5-xxl"
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [8]:
tokenized_inputs = concatenate_datasets([dataset['train'], dataset['test']]).map(lambda x: tokenizer(x["dialogue"],truncation = True),batched = True,remove_columns = ['dialogue','summary'])

Map:   0%|          | 0/15551 [00:00<?, ? examples/s]

In [9]:
input_lengths = [len(x) for x in tokenized_inputs["input_ids"]]

In [10]:
import numpy as np
import pandas as pd

In [11]:
max_source_length = int(np.percentile(input_lengths,85))
print(f"Max Source Length : {max_source_length}")

Max Source Length : 255


In [12]:
tokenized_target = concatenate_datasets([dataset['train'], dataset['test']]).map(lambda x: tokenizer(x["summary"], truncation = True),batched = True, remove_columns = ["dialogue","summary"])

Map:   0%|          | 0/15551 [00:00<?, ? examples/s]

In [13]:
output_lengths = [len(x) for x in tokenized_target["input_ids"]]
max_output_length = int(np.percentile(output_lengths,90))
print(f"Max Target Length : {max_output_length}")

Max Target Length : 50


In [14]:
def preprocess_function(sample, padding = "max_length"):
    inputs = ["summarize :" + dialogue for dialogue in sample["dialogue"]]
    model_inputs = tokenizer(inputs, max_length = max_source_length, padding = padding, truncation =True)
    labels = tokenizer(text_target = sample["summary"], max_length = max_output_length, padding =padding, truncation = True)

    if padding =="max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [15]:
tokenized_datasets = dataset.map(preprocess_function, batched =True, remove_columns = ["dialogue","summary","id"])

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [19]:
model_id = "google/flan-t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_id,device_map = "auto")

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [21]:
from peft import LoraConfig, get_peft_model, TaskType

In [22]:
lora_config = LoraConfig(
    r = 16,
    lora_alpha = 32,
    target_modules=["q","v"],
    lora_dropout = 0.05,
    bias = "none",
    task_type = TaskType.SEQ_2_SEQ_LM
)

In [23]:
model = get_peft_model(model,lora_config)

In [25]:
model.print_trainable_parameters()

trainable params: 1,769,472 || all params: 249,347,328 || trainable%: 0.7096


In [26]:
from transformers import DataCollatorForSeq2Seq

In [27]:
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = model, label_pad_token_id=-100,pad_to_multiple_of=8)

In [28]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [31]:
training_args = Seq2SeqTrainingArguments(
    output_dir = "summarization_model",
    auto_find_batch_size = True,
                                         learning_rate = 1e-3,
                                         num_train_epochs = 5,
                                         report_to = "tensorboard"
                                         )

In [32]:
trainer = Seq2SeqTrainer(
    model =model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"]
)

In [33]:
model.config.use_cache = False

In [34]:
trainer.train()

Step,Training Loss
500,1.560100
1000,1.563400
1500,1.569100


Step,Training Loss
500,1.560100
1000,1.563400
1500,1.569100
2000,1.530900
2500,1.510000
3000,1.520700
3500,1.499500
4000,1.463200
4500,1.455700
5000,1.467000


TrainOutput(global_step=9210, training_loss=1.4615535176926404, metrics={'train_runtime': 4363.191, 'train_samples_per_second': 16.882, 'train_steps_per_second': 2.111, 'total_flos': 2.541981446701056e+16, 'train_loss': 1.4615535176926404, 'epoch': 5.0})

In [36]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.5 MB/s eta 0:00:00


In [37]:
import evaluate

In [38]:
peft_model_id = "result"

trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)

('result/tokenizer_config.json',
 'result/special_tokens_map.json',
 'result/spiece.model',
 'result/added_tokens.json',
 'result/tokenizer.json')

In [39]:
#### EVALUATION #########

In [40]:
import torch
from peft import PeftModel, PeftConfig

In [41]:
config = PeftConfig.from_pretrained(peft_model_id)

In [42]:
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, device_map = "auto")

In [43]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [45]:
model = PeftModel.from_pretrained(model, peft_model_id)
model.eval()

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): PeftModelForSeq2SeqLM(
      (base_model): LoraModel(
        (model): T5ForConditionalGeneration(
          (shared): Embedding(32128, 768)
          (encoder): T5Stack(
            (embed_tokens): Embedding(32128, 768)
            (block): ModuleList(
              (0): T5Block(
                (layer): ModuleList(
                  (0): T5LayerSelfAttention(
                    (SelfAttention): T5Attention(
                      (q): lora.Linear(
                        (base_layer): Linear(in_features=768, out_features=768, bias=False)
                        (lora_dropout): ModuleDict(
                          (default): Dropout(p=0.05, inplace=False)
                        )
                        (lora_A): ModuleDict(
                          (default): Linear(in_features=768, out_features=16, bias=False)
                        )
                        (lora_B): ModuleDict(
                          (default): 

In [48]:
from random import randrange

In [50]:
sample = dataset['test'][randrange(len(dataset['test']))]
input_ids = tokenizer(sample['dialogue'], return_tensors = "pt",truncation = True).input_ids.cuda()
outputs = model.generate(input_ids = input_ids, max_new_tokens = 50, top_p = 0.9)
print(f"input sentence : {sample['dialogue']}")
print(f"Summary :{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens = True)[0]}")

input sentence : Lincoln: Heeyyy ;* whats up
Fatima: I talked to Jenson, he’s not too happy ;p
Lincoln: the place sucks??
Fatima: No, the place is ok, I think, we can go there, it’s about Alene
Lincoln: typical, dont worry about it
Fatima: He thinks she may have a depression :[
Lincoln: nothin new, everyone has it, she needs a doctor then
Fatima: But she won’t go ;/
Lincoln: so she’s destroying her life fuck it its not your problem
Fatima: It is, they’re both my friends!
Lincoln: you better think what to do if they break up
Fatima: Ehh yes Ill have a problem ;//
Lincoln: both blaming each other and talking with you about it, perfect
Fatima: Alene is just troubled… She’d been through a lot…
Lincoln: everyone has their problems, the question is are ya doin sth about them
Fatima: She has problems facing it, don’t be surprised :[
Lincoln: then it is her problem
Fatima: You are so cruel at times… o.O
Lincoln: maybe, for me its just a common sense
Fatima: Why can’t everyone be just happy???


In [51]:
import evaluate
import numpy
from datasets import load_from_disk
from tqdm import tqdm

In [53]:
!pip install rouge-score -q

  Preparing metadata (setup.py) ... done


In [63]:
metric = evaluate.load("rouge")

In [87]:
def evaluate_peft_model(sample, max_target_length = 50):
    input_ids = tokenizer(sample["dialogue"], return_tensors = "pt", truncation = True, max_length = max_source_length).input_ids.cuda()
    outputs= model.generate(input_ids = input_ids, max_new_tokens = max_target_length, top_p = 0.9)
    predictions = tokenizer.decode(outputs[0].detach().cpu().numpy(), skip_special_tokens = True)
    #labels = np.where(sample['labels'] != -100, sample['labels'], tokenizer.pad_token_id)
    labels = sample["summary"]
    return predictions, labels

In [65]:
dataset = load_dataset("samsum")

In [88]:
predictions = []
refrences = []

for sample in tqdm(dataset['test']):
    p,l = evaluate_peft_model(sample)
    predictions.append(p)
    refrences.append(l)

  0%|          | 0/819 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 819/819 [09:13<00:00,  1.48it/s]


In [74]:
dataset['test'][0]

{'id': '13862856',
 'dialogue': "Hannah: Hey, do you have Betty's number?\nAmanda: Lemme check\nHannah: <file_gif>\nAmanda: Sorry, can't find it.\nAmanda: Ask Larry\nAmanda: He called her last time we were at the park together\nHannah: I don't know him well\nHannah: <file_gif>\nAmanda: Don't be shy, he's very nice\nHannah: If you say so..\nHannah: I'd rather you texted him\nAmanda: Just text him 🙂\nHannah: Urgh.. Alright\nHannah: Bye\nAmanda: Bye bye",
 'summary': "Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry."}

In [89]:
rouge = metric.compute(predictions = predictions, references = refrences, use_stemmer = True)

In [91]:
print(f"Rogue1: {rouge['rouge1']* 100:2f}%")
print(f"rouge2: {rouge['rouge2']* 100:2f}%")
print(f"rougeL: {rouge['rougeL']* 100:2f}%")
print(f"rougeLsum: {rouge['rougeLsum']* 100:2f}%")

Rogue1: 46.955498%
rouge2: 22.674003%
rougeL: 38.789827%
rougeLsum: 38.789798%
